In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, SentencesDataset, losses, CrossEncoder
from torch.utils.data import DataLoader
from sentence_transformers.readers import InputExample

2022-03-31 09:17:39.104538: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-31 09:17:39.104572: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import utils
GLOBAL_VARS = utils.get_global_vars()
train_df = pd.read_csv(f"{GLOBAL_VARS['dataset_path']}/train-balanced.csv", sep='|', header=None)
train_df = train_df.rename(columns={0: 'sentence1', 1: 'sentence2', 2: 'label'})
test_df = pd.read_csv(f"{GLOBAL_VARS['dataset_path']}/test-balanced.csv", sep='|', header=None)

In [4]:
import json
comments = json.load(open(f'{GLOBAL_VARS["dataset_path"]}/comments.json'))
comments

{'7u4r6': {'text': 'Upvote For Simultaneous "Million Person" Marches on Wall Street And D.C.',
  'author': '[deleted]',
  'score': 48,
  'ups': 104,
  'downs': 56,
  'date': '2009-02',
  'created_utc': 1233540251,
  'subreddit': 'Economics'},
 'c07ewjj': {'text': 'Economics (29654 subscribers)',
  'author': 'pfft',
  'score': 14,
  'ups': 14,
  'downs': 0,
  'date': '2009-02',
  'created_utc': 1233549003,
  'subreddit': 'Economics'},
 '7u4a5': {'text': "Children in the Czech Republic are happier and better taken care of than in the USA or The UK - UNICEF (PDF) Austrailia didn't even make the top 20.",
  'author': '[deleted]',
  'score': 29,
  'ups': 48,
  'downs': 19,
  'date': '2009-02',
  'created_utc': 1233533923,
  'subreddit': 'worldnews'},
 'c07ey0j': {'text': 'Of course it\'s a "less of a country", those people don\'t speak English they don\'t even speak a respectable (nordic or romance) language, they speak some unintelligible "Eastern European" language, meaning of course they

In [23]:
def get_keys_of_interest_from_dict(dictionary, keys_of_interest):
    filtered_dict = {key: dictionary[key] for key in keys_of_interest if key in dictionary}
    return filtered_dict

def get_top_comment_info(top_comment):
    keys_of_interest = [
        'text',
        'author',
        #'subreddit',
        'score',
        #'ups',
        #'downs',
        #'date',
        #'created_utc'
    ]
    top_comment_info = get_keys_of_interest_from_dict(comments[top_comment], keys_of_interest)
    return top_comment_info

def get_target_comment_info(target_comment):
    keys_of_interest = [
        'text',
        'author',
        #'subreddit',
        'score',
        #'ups',
        #'downs',
        #'date',
        #'created_utc'
    ]
    target_comment_info = get_keys_of_interest_from_dict(comments[target_comment], keys_of_interest)
    return target_comment_info


def translate_line(row):
    top_comment = row[0]
    target_comment1 = row[1].split()[0]
    target_comment2 = row[1].split()[1]
    top_comment_info = get_top_comment_info(top_comment)
    target_comment1_info = get_target_comment_info(target_comment1)
    target_comment2_info = get_target_comment_info(target_comment2)
    subreddit = comments[top_comment]['subreddit'] 

    line_dict = dict()
    info_list = [top_comment_info, target_comment1_info, target_comment2_info]
    name_list = ['comment', 'answer1', 'answer2']

    for info_dict, name in zip(info_list, name_list):
        for key, info in info_dict.items():
            line_dict[f'{name}_{key}'] = info
    line_dict['subreddit'] = subreddit
    return line_dict 

In [25]:
new_train_df = train_df.iloc[:1000,:].apply(translate_line, axis=1)

KeyError: '7zp51 c07uja7 c07umjb'

In [ ]:
new_train_df

In [9]:
comments[train_df.iloc[0][1].split()[0]]

{'text': 'Religion must have the answer',
 'author': 'Reedzit',
 'score': 3,
 'ups': 3,
 'downs': 0,
 'date': '2009-02',
 'created_utc': 1233605181,
 'subreddit': 'science'}

In [3]:
model = CrossEncoder('microsoft/deberta-base',num_labels=3)

Downloading:   0%|          | 0.00/474 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/533M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'classifi

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]